<h1 style="color:black" align="center">Семинар №9</h1>

<h1 style="color:#008B8B">1. Градиентный бустинг</h1>

На лекциях обсуждался градиентный бустинг - один из самых мощных методов в машинном обучении. Он позволяет воспользоваться выразительной силой решающих деревьев и при этом контролировать их переобучение. Ниже мы пройдём по основным блокам градиентного бустинга и поймём, почему они устроены именно так.

Для начала вспомним основные принципы градиентного бустинга. Будем искать алгоритм, оптимизирующий некоторую дифференцируемую функцию потерь $L(y, z)$, в виде взвешенной суммы базовых алгоритмов:

$\large a_N(x)
=
\sum\limits_{n = 0}^{N}
    \gamma_n b_n(x).$

Как правило, веса $\gamma_n$ не подбираются и полагаются равными единице (поскольку, как правило, в деревьях всё равно потом тщательно подбираются прогнозы в листьях), но пока будет рассматривать общий случай.

Идея бустинга заключается в последовательном построении алгоритмов, каждый из которых учитывает ошибки построенной до сих пор композиции:

$\large \sum\limits_{i = 1}^{\ell}
    L(y_i, a_{N - 1}(x_i) + \gamma_N b_N(x_i))
\to
\underset{b_N, \gamma_N}{\text{min}}$


После выбора каких-нибудь простых $\gamma_0$ и $b_0(x)$ (например, для задачи регрессии можно положить $\gamma_0 = 1$ и $b_0(x) = \frac 1\ell \sum\limits_{i=1}^\ell y_i$) все последующие базовые алгоритмы стараются приблизить антиградиент функционала ошибки, взятый в точках $z = a_{N - 1}(x_i)$:

$\large s_i
=
-
\left.
\frac{\partial L(y_i, z)}{\partial z}
\right|_{z = a_{N - 1}(x_i)}$


При этом приближается антиградиент с точки зрения квадратичной функции потерь:

$\large b_N(x)
=
\underset{b \in \mathbb{A}}{\text{argmin}}
    \sum\limits_{i = 1}^{\ell}
        \left(
            b(x_i) - s_i
        \right)^2$


Подбор коэффициентов производится просто через задачу одномерной оптимизации:

$\large \gamma_N
=
\underset{\gamma \in \mathbb{R}}{\text{argmin}}
\sum\limits_{i=1}^{\ell}
L(y_i, a_{N-1}(x_i) + \gamma b_N (x_i))$


Обсудим теперь подробнее некоторые моменты.

<h2 style="color:#008B8B">1.1 Почему градиентный бустинг устроен именно так?</h2>

### Зачем сдвиги в бустинге считаются через производные функции потерь?

Почему нельзя использовать сдвиги вида $y_i - a_{N - 1}(x_i)$? Казалось бы, если удастся хорошо приблизить эти отклонения с помощью $b_N(x)$, то будет выполнено $a_{N-1}(x_i) + b_N(x_i) \approx y_i$.

Мы решаем сложные задачи, в которых, скорее всего, ещё и не идеальные данные, есть шумы, выбросы и так далее. Это означает, что не стоит рассчитывать на получение непереобученной модели с нулевой ошибкой на обучающей выборке. Мы это понимаем, и поэтому через функцию потерь определяем, какие ошибки более приемлемы, чем другие. Если использовать сдвиги, равные разности правильного ответа и прогноза текущей композиции, то мы полностью игнорируем эти требования. Разберём несколько примеров.

**Пример 1.** Начнём с регрессии. Допустим, у нас несимметричная функция потерь, где мы сильнее штрафуем за завышение прогноза:

$\large L(y, z)
=
\frac{1}{2}
(10 [z \geq y] + [z < y])
(y - z)^2$

Рассмотрим два объекта $x_1$ и $x_2$ с правильными ответами $y_1 = y_2 = 0$ и прогнозами текущей композиции $a_{N - 1}(x_1) = 5$ и $a_{N - 1}(x_2) = -5$. Если взять сдвиги, равные $y_i - a_{N - 1}(x_i)$, то базовая модель будет пытаться одинаково скорректировать прогнозы на обоих объектах. Но, конечно, надо сильнее сконцентрироваться на коррекции прогноза на первом объекте, поскольку на нём штраф больше: $L(0, 5) = 125 > 12.5 = L(0, -5)$. Сдвиги, посчитанные через частные производные, лучше отражают приоритеты: $s_1 = -50$, $s_2 = 5$. При этом не страшно, что на первом объекте мы сделаем слишком большую корректировку - это будет исправлено с помощью веса $\gamma_N$ базовой модели или с помощью других техник.

**Пример 2.** Обсудим теперь обучение на абсолютную ошибку $L(y, z) = |y - z|$.
Вычислим для неё сдвиги:

$\large s_i = - \left. \frac{\partial |y_i - z|}{\partial 
    z} \right|_{z=a_{N-1}(x_i)} = 
\Bigl. \text{sign}(y_i - z) \Bigr|_{z=a_{N-1}(x_i)} =
\text{sign}(y_i - a_{N-1}(x_i))$

И это снова более логично, чем обучение на $y_i - a_{N - 1}(x_i)$! Мы знаем, что с точки зрения квадратичной ошибки даже небольшое уменьшение серьёзной ошибки оказывается лучше, чем доведение небольшой ошибки до нуля:

$\large L(0, 1000) - L(0, 999) = 1999 > 1 = L(0, 1) - L(0, 0).$

Как мы видим, при уменьшении прогноза тысячи на еденицу уменьшает ошибку на 1999, что оказывается лучше, чем доведение небольшой ошибки до нуля, так как ошибка изменится всего на еденицу. 

Поэтому для квадратичной ошибки разумно использовать сдвиги $y_i - a_{N - 1}(x_i)$, поскольку чем больше отклонение от правильного ответа, тем больше штраф и тем сильнее надо корректировать прогнозы. У абсолютной ошибки такого свойства нет, изменения штрафа не зависят от отклонения прогноза от факта. Поэтому независимо от того, на каком объекте мы приблизим прогноз к правильному ответу на единицу, средняя абсолютная ошибка улучшится на одну и ту же величину. У нас как раз получилось, что сдвиги на всех объектах одинаковы по модулю и отличаются лишь знаком - это соответствует описанному выше свойству абсолютной ошибки.

**Пример 3.** Перейдём к классификации. Если у нас два класса ($\mathbb{Y} = \{-1, +1\}$), и сама композиция также возвращает прогноз из $\mathbb{Y}$, то отклонения всегда будут принимать значения из небольшого множества: $|y_i - a_{N - 1}(x_i)| \in \{0, +2\}$. 


Получается, что если модель уже выдаёт правильный ответ, то сдвиги будут равню нулю, и никакие модификации для этого объекта вноситься не будут. Разумеется, это нас не устраивает - как правило, функции потерь в классификации стараются максимизировать отступ (или хотя бы довести его до определённого положительного значения), а полученные нами сдвиги никак не зависят от отступа.

Чтобы это исправить, договоримся, что наша композиция $a_N(x)$ будет выдавать вещественные числа, и по смыслу они будут являться оценками *логита*, то есть логарифма отношения вероятности положительного класса к вероятности отрицательного класса:

$\large a_N(x)
=
\log
\frac{
    p(y = +1 | x)
}{
    1 - p(y = +1 | x)
}$

Если выразить отсюда вероятность, то получится

$\large p(y = +1 | x)
=
\sigma\left(
    a_N(x)
\right)
=
\frac{
    1
}{
    1 + \exp(-a_{N}(x))
}$

То есть сигмоида от модели будет оценивать вероятность положительного класса. Эту же идею мы использовали при обучении линейных классификаторов - там скалярное произведение как раз оценивало логиты. Получаем, что $a_N(x)$ - чем больше, тем выше уверенность в том, что объект положительный. С таким подходом обучение на $y_i - a_{N - 1}(x_i)$ выглядит ещё менее логичным, поскольку в этом случае мы будем пытаться уменьшить отступ там, где он большой положительный, то есть по сути будем запрещать модели быть уверенной в своём ответе. 

Например, имеется положительный равный $y_i = +1$ и построенна композиция выдает ответ на этом объекте $a_{N-1} (x_i) = 10000$, следовательно модель уверена в том, что этот объект положителен, у нее большой отступ на этом объекте. Отступ на этом объекте будет равен $y_i - a_{N-1} = -9999$, сдвиг говорит о том, что необходимо уменьшать прогноз модели, так как выход модели должен быть равен $+1$, а мы выдали $10000$. Следовательно, если использовать данные сдвиги, они будут запрещать строить уверенные модели, если модель уверена на некотором объекте, сдвиги будут уменьшать отступ объекта.

В завершение посчитаем сдвиги для логистической функции потерь:
$L(y, z) = \log (1 + \exp(-yz))$. Для этого вспомним, что логистическая функция потерь выражается через сигмоиду $\sigma(x) = \frac{1}{1 + \exp(-x)}$ следующим образом:

$\large L(y, z) = \log \left( \frac{1}{\sigma(yz)} \right) = -\log \sigma(yz)$

Далее, пользуясь формулой для производной сигмоиды $\sigma'(x) = \sigma(x)(1 - \sigma(x))$, получаем

\begin{align*}
    s_i &= \left. \frac{\partial \log \sigma(y_i z)}{\partial z} 
        \right|_{z=a_{N-1}(x_i)} =
    \left. \frac{1}{\sigma(y_i z)} \sigma(y_i z) (1 - \sigma(y_i z)) y_i
        \right|_{z=a_{N-1}(x_i)} =\\
    &= (1-\sigma(y_i a_{N-1}(x_i))) y_i =
    \frac{y_i}{1 + \exp(y_i a_{N-1}(x_i))}.
\end{align*}

Заметим, что чем больше отступ, тем меньше будут сдвиги по модулю. Здесь числитель говорит в какую сторону необходимо двигаться, для положитльных объектов числитель и знаменатель будет положителен, поэтому модель будет стремиться увеличивать прогноз композиции, для отрицательных объектов данная дробь будет отрицательной, поэтому прогноз композиции будет уменьшаться в сторону минус бесконечности. А знаметель говорит о том, чем больше отступ, тем меньше будет $|s_i|$. То есть, знаменатель говорит о том, что если на объекте уже хороший отступ, тогда сильно нет необходимости что-то изменять.

### Почему базовая модель приближает сдвиги по MSE, а не по исходной функции потерь?

Казалось бы, надо везде, где можно, использовать исходную функцию потерь,
в том числе и для обучения базовой модели:

$\large \sum\limits_{i = 1}^{\ell}
    L(s_i, b_N(x_i))
\to
\min\limits_{b_N}$

Функция потерь задаёт штрафы для прогнозов моделей. При этом сдвиги $s_i$ уже содержат в себе информацию о том, в какую сторону и как сильно надо корректировать прогнозы композиции, и будет разумно аппроксимировать их
с одинаковой важностью для всех объектов. Если же и здесь использовать исходную функцию потерь, то иногда можно получить странные эффекты. Делаем выводы, что исходна фукнция потерь задает требования к решению исходной задачи, она скорее всего непригодна для измерения отклонений остатков, так как смысл задачи другой.

Получаем, что в бустинге две фукнции потерь, исходная, которая используется для рассчета градинета и ещё одна для обучения базовой модели на градиент.

Вернёмся к несимметричной функции потерь:

$\large L(y, z)
=
\frac{1}{2}
(10 [z \geq y] + [z < y])
(y - z)^2$

У нас было два объекта с правильными ответами $y_1 = y_2 = 0$, прогнозами текущей композиции $a_{N - 1}(x_1) = 5$ и $a_{N - 1}(x_2) = -5$ и сдвигами $s_1 = -50$, $s_2 = 5$. Наша функция потерь будет сильнее поощрять алгоритмы, которые выдадут на первом объекте корректировку сильнее $-50$, а на втором объекте - до $5$. Но это не имеет никакого смысла, поскольку дерево лишь должно выучить, уменьшаем или увеличиваем мы выход композиции на каждом объекте, и изменение на первом объекте должно быть больше относительно изменения на втором объекте.

В случае с классификацией всё ещё хуже. Например, если мы работаем с логистической функцией потерь,

$L(y, z) = \log(1 + \exp(-yz))$

то при обучении одной модели на классы она имеет смысл правдоподобия, а также заставляет модель корректно оценивать вероятности классов, следовательно функция потерь хорошо интерпретируема если $y \in \{-1, +1\}$. Если же в неё вместо правильных классов $y$ подставить сдвиги $s_i \in \mathbb{R}$, то оптимизационную задачу будет достаточно сложно проинтерпретировать.

Заметим, что в то же время существуют способы сильнее учитывать исходную функцию потерь в процессе обучения базовых моделей - мы разберём их, когда будем обсуждать XGBoost и связанные с ним модификации классического алгоритма.

Отметим также, что минимизация MSE имеет смысл в бустинге по ещё одной причине. Распишем этот функционал:


$\large \sum\limits_{i = 1}^{\ell}
    (b_N(x_i) - s_i)^2
=
\sum\limits_{i = 1}^{\ell}
    (b_N^2(x_i)
    -
    2 s_i b_N(x_i)
    +
    s_i^2).$

Последнее слагаемое не зависит от модели и поэтому никак не повлияет на решение оптимизационной задачи. Остаются два слагаемых:

$\large \sum\limits_{i = 1}^{\ell}
    (b_N^2(x_i)
    -
    2 s_i b_N(x_i))
=
\sum\limits_{i = 1}^{\ell}
    b_N^2(x_i)
-
2
\|s\|
\|b_N(x)\|
\cos(s, b_N(x))
\to
\min\limits_{b_N},$

где $\|s\| = \sqrt{\sum\limits_{i = 1}^{\ell} s_i^2}$, $\|b_N(x)\| = \sqrt{\sum\limits_{i = 1}^{\ell} b_N^2(x_i)}$, $\cos(s, b_N(x))$ - косинус угла между соответствующими векторами. Получается, что мы (с некоторыми оговорками) требуем от модели, чтобы она угадала направление корректировки прогнозов на всей обучающей выборке.

### Почему нельзя просто обучить одно глубокое решающее дерево?

Понятно, что если делать это неаккуратно, то дерево переобучится, но наверняка ведь можно контролировать это?

Действительно, так можно делать. Но как контролировать переобучение? Можно ограничить глубину дерева или минимальное число объектов в листе, но это слишком просто - не факт, что эти критерии гарантируют хороший результат на тестовой выборке. Это означает, что во время построения надо контролировать ошибку на тесте, и с учётом неё принимать решение об остановке построения дерева в каждой вершине. Более того, из-за жадной природы алгоритм построения дерева может выбирать некорректные разбиения в начале - если мы поймём, что именно из-за них случается переобучение, может потребовать радикально перестраивать дерево. В итоге получится громоздкий и трудоёмкий алгоритм.

Бустинг же позволяет балансировать между сложностью и качеством итоговой модели. На каждом шаге он строит очень небольшие деревья, которые не могут привести к сильной подгонке под обучающую выборку. При этом после добавления каждого дерева, как правило, проверяется ошибка на тестовых данных, и если начинается переобучение, то процедура построения композиции прекращается.

Делаем выводы, что с глубокими деревьями сложнее конролировать сложность композиции из-за чего можно переобучиться.

### Можно ли задействовать идеи из модификаций градиентного спуска?

Да, можно!
Дальше в курсе мы разберём, как можно задействовать вторые производные функции потерь для имитации оптимизации второго порядка. Но также можно использовать и другие подходы - например, из метода инерции.

Предлагается адаптировать ускоренный метод инерции Нестерова (Nesterov momentum). Разберём сначала этот метод. Напомним, что обычный метод инерции устроен так:

\begin{align*}
    &h_0 = 0;\\
    &h_k = \alpha h_{k - 1} + \eta_k \nabla_w Q(w^{(k-1)});\\
    &w^{(k)} = w^{(k-1)} - h_k.
\end{align*}

Заметим, что мы ещё до вычисления $h_k$ знаем, что сдвинемся в направлении $h_{k - 1}$, немного скорректированном на антиградиент функционала ошибки в точке $w^{(k-1)}$. Воспользуемся этим наблюдением и сначала сдвинемся по $h_{k - 1}$, а затем вычислим антиградиент уже в этой точке и шагнём по нему:

\begin{align*}
    &h_0 = 0;\\
    &h_k = \alpha h_{k - 1} + \eta_k \nabla_w Q(w^{(k-1)} - \alpha h_{k - 1});\\
    &w^{(k)} = w^{(k-1)} - h_k.
\end{align*}

Теперь перенесём этот метод на градиентный бустинг. Помимо самой композиции $a_{N}(x)$ будем теперь строить <<инерционную>> модель $h_N(x)$, которая будет как бы накапливать антиградиенты. В начале инициализируем её так же, как и саму композицию: $h_0(x) = a_0(x) = b_0(x)$. Допустим, мы сделали $N - 1$ шаг градиентного бустинга и построили $a_{N - 1}(x)$ и $h_{N - 1}(x)$. 

Необходимо посчитать сдвиги, до этого сдвиги считались как градиенты по прогнозу модели, но метот Nesterov momentum сообщает, что градиенты необходимо считать в такой модифицированной точке $w^{(k-1)} - \alpha h_{k - 1}$, где аналогом $w^{k-1}$ будет выступать выход текущей модели $a_{N-1}(x_i)$, и ещё необходимо скорректировать на $h_{k-1}$ и это будут выходы инерционной модели $h_{N-1}(x_i)$. Базовая модель $b_N(x)$ должна аппроксимировать антиградиент - значит, теперь будем обучать её
на сдвиги вида:

$\large s_i
=
-
\left.
\frac{\partial L(y_i, z)}{\partial z}
\right|_{z = a_{N - 1}(x_i) + \alpha h_{N - 1}(x_i)}$

Единственное отличие, что в модифицированной точке у нас стоял минус, а тут при нахождении градиента для сдвига стоит плюс, так как мы договорились, что $h_N$ накапливает антиградиенты. До этого формуле была сумма градиентов, поэтому брали с минусом, а здесь мы уже накапливаем градиенты с минусом, поэтому ставим плюс.

Видно, что здесь в качестве обучаемых параметров у нас выступают выходы уже построенной композиции $a_{N - 1}(x_i)$, а в качестве инерции - выходы функции $h_{N - 1}(x_i)$. Поле этого мы обучаем $b_N$ на $s_i$ и теперь мы можем обновить инерционную функцию и саму композицию:


$\large h_N(x)
=
\alpha h_{N - 1}(x)
+
\eta
b_N(x)$

$\large a_N(x)
=
a_{N - 1}(x)
+
h_N(x).$

В статье используется немного модифицированная версия, где параметр $\alpha$ меняется в зависимости от номера итерации и влияет также на длину шага $\eta$. Помимо этого отмечается, что указанные формулы работают не лучшим образом, поскольку в инерционной функции накапливается ошибка, связанная с неточным приближением антиградиента базовыми функциями. Это исправляется усложнённой процедурой построения инерционной функции - но это детали, а главное, что идеи из оптимизация действительно удаётся перенести и на градиентный бустинг, что открывает большой простор для улучшений.

**Проблема экстраполяции (подробнее в методичке)**

Деревья не умеют экстрополировать, а бывают задачи, где меняется распределение признако на test и с этим необходимо работать. В таких случаях можно обучить линейную модель $c(x)$, она может получить плохой, но она способна экстраполировать, дальше считаем отклонение этой модели от правильного ответа $y_i - c(x_i)$ и на это обучаем бустинг. Следовательно бустинг работает на корректировку линейной модели.

**Доработка линейных моделей**

Существует метод mars, где в каждом листе дерева линейная модель.